In [95]:
import csv
import statistics

dis_class = set()
tsv_file = open('../output/instance_prediction_datasets/P161.objects.1250.tsv',encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    dis_class.add(line[0])
print("distinct classes: ", len(dis_class))

print("retrieving correct answers...")
correct_answers = getCorrectAnswers(list(dis_class))
guesses = {}
for elem in dis_class:
    guesses[elem] = "Q5"

print("retrieving scores...")
scores = []
for elem in guesses:
    this_score = score(guesses[elem], correct_answers[elem])
    scores.append(this_score)
print("final avg. score: ", statistics.mean(scores))

distinct classes:  1250
retrieving correct answers...
retrieving scores...
final avg. score:  0.996


In [23]:
# score P31 datasets
import csv

direct_subclasses = {}
direct_superclasses = {}

tsv_file = open('../derived.P279.tsv/derived.P279.tsv',encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[1] not in direct_superclasses:
        direct_superclasses[line[1]] = [line[3]]
    else:
        direct_superclasses[line[1]].append(line[3])
    if line[3] not in direct_subclasses:
        direct_subclasses[line[3]] = [line[1]]
    else:
        direct_subclasses[line[3]].append(line[1])

In [93]:
import mmap

def getCorrectAnswers(candidates):
    answers = {}
#     with open('../derived.P31.tsv/derived.P31.tsv',encoding="utf-8") as f:
#         s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
#         while len(candidates) != 0:
#             line = s.readline()
#             for candidate in candidates:
#                 if str.encode("\t"+candidate+"\t") in line:
#                     str1 = line.decode("utf-8")
#                     split = str1.split("\t")
#                     answer = split[-1].split("\n")[0]
#                     answers[candidate] = answer
#                     candidates.remove(candidate)
    tsv_file = open('../output/allConstraintsAnalysis_Final/instances/instances.objects.P161.tsv', encoding="utf-8")
    read_tsv = csv.reader(tsv_file, delimiter="\t")
    for line in read_tsv:
        answers[line[0]] = line[1]
    return answers

In [77]:
def score (guess, correct):
    if guess == correct: 
        return 1
    else:
        subclasses = []
        superclasses = []
        if guess in direct_subclasses:
            subclasses = direct_subclasses[guess]
        if guess in direct_superclasses:
            superclasses = direct_superclasses[guess]
        for i in range(1,6):
            new_subclasses = []
            new_superclasses = []
            for subc in subclasses:
                if subc == correct:
                    if i <= 3:
                        return .7**i
                    else:
                        return .8**i
                else:
                    if subc in direct_subclasses:
                        for new_sub in direct_subclasses[subc]:
                            new_subclasses.append(new_sub)
            for supc in superclasses:
                if supc == correct:
                    if i <= 3:
                        return .7**i
                    else:
                        return .8**i
                else:
                    if supc in direct_superclasses:
                        for new_sup in direct_superclasses[supc]:
                            new_superclasses.append(new_sup)
            subclasses = new_subclasses
            superclasses = new_superclasses
    return 0